## find the week and month having peak (highest) number of active cases for wave-1 and wave-2

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta 
import warnings
warnings.filterwarnings('ignore')

### LOADING FILES

In [2]:
covid_cases = pd.read_csv('districts.csv',parse_dates=['Date'],usecols=['State','District','Date','Confirmed'],index_col=False)

In [3]:
vacc_df1 = pd.read_csv('cowin_vaccine_data_districtwise.csv',usecols=['State_Code','State'],index_col=False).drop_duplicates(subset=None, keep='first', inplace=False)
covid_cases = covid_cases[covid_cases['Date'] <= '2021-08-14']

In [4]:
#DROPPING NA
vacc_df1.dropna(inplace=True)

AS MONTH STARTS FROM 15TH TO 14TH

In [5]:
covid_cases['Date_1'] = covid_cases["Date"] + timedelta(days=-14)

CONVERTING DATE AS AN INTEGER TO EASE GROUPING

In [6]:
covid_cases['dateid'] = covid_cases.groupby(['Date_1']).ngroup()+1
covid_cases['monthid'] = pd.DatetimeIndex(covid_cases['Date_1']).month + 100*pd.DatetimeIndex(covid_cases['Date_1']).year

In [7]:
#THIS IS TO ENSURE WEEK STARTS FROM SUNDAY SO WE TAKE A SUNDAY
covid_cases['diff1']=pd.to_datetime(covid_cases['Date'])-pd.to_datetime('2020-01-05')
covid_cases['day_id1'] = ((covid_cases['diff1']/ np.timedelta64(1, 'D'))/7 ).astype(int)

covid_cases['diff2']=(pd.to_datetime(covid_cases['Date'])-pd.to_datetime('2020-01-05'))/np.timedelta64(1, 'D')-4
covid_cases['dayid2'] = (covid_cases['diff2']/7 ).astype(int)

GROUPING

In [8]:
covid_cases['weekid_1'] = covid_cases.groupby(['day_id1']).ngroup() + 1
covid_cases['weekid_2'] = covid_cases.groupby(['dayid2']).ngroup() + 1

In [9]:
covid_cases['monthid'] = covid_cases.groupby(['monthid']).ngroup() + 1 

In [10]:
df1 = pd.merge(vacc_df1,covid_cases,how='inner',left_on=['State'],right_on = ['State'])
df1['districtid']=df1['State_Code']+"_"+df1["District"]

In [11]:
df1_month = df1[['State_Code','districtid','monthid','Confirmed']]
df1_weekid1 = df1[['State_Code','districtid','weekid_1','Confirmed']]
df1_weekid2 = df1[['State_Code','districtid','weekid_2','Confirmed']]
#df1_overall = df1[['State_Code','districtid','Confirmed']]

TAKING MAX VALUE OF A MONTH/WEAK

In [12]:
df1_month = df1_month.groupby(['State_Code','monthid','districtid'])['Confirmed'].max().reset_index(name='Confirmed') 

In [13]:
df1_sub1 = df1_weekid1.groupby(['State_Code','weekid_1','districtid'])['Confirmed'].max().reset_index(name='Confirmed') 

In [14]:
df1_sub2 = df1_weekid2.groupby(['State_Code','weekid_2','districtid'])['Confirmed'].max().reset_index(name='Confirmed')
 

In [15]:
#AS DATA IS CUMULATIVE WE PERFORM DIFF (2nd ROW - 1ST ROW and so on )

In [16]:
df1_sub1['Cases'] = df1_sub1.groupby(['districtid'])['Confirmed'].diff().fillna(df1_sub1['Confirmed'])
df1_sub2['Cases'] = df1_sub2.groupby(['districtid'])['Confirmed'].diff().fillna(df1_sub2['Confirmed'])
df1_month['Cases'] = df1_month.groupby(['districtid'])['Confirmed'].diff().fillna(df1_month['Confirmed'])

In [17]:
df1_sub1 = df1_sub1[['State_Code','weekid_1','districtid','Cases']]
df1_sub2 = df1_sub2[['State_Code','weekid_2','districtid','Cases']]

In [18]:
df1_sub1['weekid_1'] = df1_sub1['weekid_1']*2
df1_sub2['weekid_2'] = df1_sub2['weekid_2']*2+1

In [19]:
df1_sub2 = df1_sub2.rename(columns={'weekid_2':'weekid_1'})

In [20]:
df1_week = pd.concat([df1_sub1,df1_sub2])

In [21]:
df1_week['weekid'] = df1_week.groupby(['weekid_1']).ngroup()+1

In [22]:
df1_week = df1_week[['State_Code','weekid','districtid','Cases']]

df1_month = df1_month[['State_Code','monthid','districtid','Cases']]

In [23]:
df1_stwise = df1_month[['State_Code','monthid','Cases']]

In [24]:
df1_stwise_month = df1_stwise.groupby(['State_Code','monthid'])['Cases'].sum().reset_index(name='Cases')

In [25]:
#coverting into integer type
df1_week['week_id'] = df1_week['weekid'].astype(int)

In [26]:
df1_week = df1_week[df1_week['week_id']!=2]
df1_week['week_id'] = np.where(df1_week['week_id']%2 == 0,df1_week['week_id']-2,df1_week['week_id'])

In [27]:
df1_week['weekid'] = df1_week['week_id'].astype(int)
df1_week = df1_week.drop(columns=['week_id'])

CALCULATING STATEWISE

In [28]:
df1_stwise=df1_week[["State_Code","weekid","districtid","Cases"]]

In [29]:
df1_stwise = df1_stwise[['State_Code','weekid','Cases']]

In [30]:
df1_stwise_week = df1_stwise.groupby(['State_Code','weekid'])['Cases'].sum().reset_index(name='Cases')

In [31]:
df1_stwise_week_1 = df1_stwise_week[df1_stwise_week['weekid'] < 40]
df1_stwise_week_2 = df1_stwise_week[df1_stwise_week['weekid'] > 80 ]

In [32]:
df1_overall1=df1_stwise_week_1[["State_Code","weekid","Cases"]]

In [33]:
df1_stwise_week_1=df1_stwise_week_1.loc[df1_stwise_week_1.groupby("State_Code")["Cases"].idxmax()]


RENAMING AND TAKING ONLY REQUIRED COLS

In [34]:
df1_stwise_week_1 = df1_stwise_week_1.rename(columns = {'weekid':'wave1 - weekid'}).drop(columns= ['Cases'])
df1_stwise_week_1 = df1_stwise_week_1[['State_Code','wave1 - weekid']]

In [35]:
df1_overall1=df1_overall1.groupby(['weekid'])['Cases'].sum().reset_index(name='Cases')


AS DATA IS CUMULATIVE WE TAKE MAX VALUE

In [36]:
date_of_week=df1_overall1
df1_overall1_max =date_of_week[date_of_week["Cases"]== date_of_week["Cases"].max()]

In [37]:
df1_overall1_max = df1_overall1_max.rename(columns = {'weekid':'wave1 - weekid'}).drop(columns= ['Cases'])
df1_overall1_max["Overall"] = "India"

In [38]:
df1_overall2=df1_stwise_week_2[["State_Code","weekid","Cases"]]

In [39]:
df1_stwise_week_2=df1_stwise_week_2.loc[df1_stwise_week_2.groupby("State_Code")["Cases"].idxmax()]

In [40]:
df1_stwise_week_2 = df1_stwise_week_2.rename(columns = {'weekid':'wave2 - weekid'}).drop(columns= ['Cases'])

In [41]:
df1_overall2=df1_overall2.groupby(['weekid'])['Cases'].sum().reset_index(name='Cases')


In [42]:
date_of_week2=df1_overall2
df1_overall2_max =date_of_week2[date_of_week2["Cases"]== date_of_week2["Cases"].max()]

In [43]:
df1_overall2_max = df1_overall2_max.rename(columns = {'weekid':'wave2 - weekid'}).drop(columns= ['Cases'])
df1_overall2_max["Overall"] = "India"

In [44]:
df1_overall_wk_peak = pd.merge(df1_overall1_max,df1_overall2_max, how="inner",on="Overall")

In [45]:
df1_sub1 = df1_week[df1_week['weekid'] < 40]
df1_sub2 = df1_week[df1_week['weekid'] > 80 ]

In [46]:
dfnew1 = df1_sub1.groupby('districtid')['Cases'].transform(max) == df1_sub1['Cases']
df1_sub1 = df1_sub1[dfnew1]


In [47]:
df1_sub1 = df1_sub1.rename(columns = {'weekid':'wave1 - weekid'}).drop(columns= ['Cases'])

In [48]:
temp = df1_sub2.groupby('districtid')['Cases'].transform(max) == df1_sub2['Cases']
df1_sub2 = df1_sub2[temp]


In [49]:
df1_sub2 = df1_sub2.rename(columns = {'weekid':'wave2 - weekid'}).drop(columns= ['Cases'])

In [50]:
df1_month_1 = df1_month[df1_month['monthid'] < 10]
df1_month_2 = df1_month[df1_month['monthid'] > 9 ]

In [51]:
df1_month_1=df1_month_1.loc[df1_month_1.groupby("districtid")["Cases"].idxmax()]

In [52]:
df1_month_1 = df1_month_1.rename(columns = {'monthid':'wave1 - monthid'}).drop(columns= ['Cases'])

In [53]:
df1_month_2=df1_month_2.loc[df1_month_2.groupby("districtid")["Cases"].idxmax()]
df1_month_2 = df1_month_2.rename(columns = {'monthid':'wave2 - monthid'}).drop(columns= ['Cases'])

In [54]:
df1_stwise_month_1 = df1_stwise_month[df1_stwise_month['monthid'] < 10]
df1_stwise_month_2 = df1_stwise_month[df1_stwise_month['monthid'] > 9 ]

In [55]:
df1_overall_month_1 = df1_stwise_month_1.groupby("monthid")["Cases"].sum().reset_index(name='Cases')
df1_overall_month_2 = df1_stwise_month_2.groupby("monthid")["Cases"].sum().reset_index(name='Cases')

In [56]:
df1_overall_month_1 = df1_overall_month_1[df1_overall_month_1["Cases"]==df1_overall_month_1["Cases"].max()]
df1_overall_month_1 = df1_overall_month_1.rename(columns = {'monthid':'wave1 - monthid'}).drop(columns= ['Cases'])
df1_overall_month_1["Overall"]="India"


In [57]:
df1_overall_month_2 = df1_overall_month_2[df1_overall_month_2["Cases"]==df1_overall_month_2["Cases"].max()]
df1_overall_month_2 = df1_overall_month_2.rename(columns = {'monthid':'wave2 - monthid'}).drop(columns= ['Cases'])
df1_overall_month_2["Overall"]="India"


In [58]:
df1_overall_month_peak = pd.merge(df1_overall_month_1,df1_overall_month_2,how="inner",on="Overall")

In [59]:
df1_overall_month_peak

,wave1 - monthid,Overall,wave2 - monthid
0,5,India,13


In [60]:
df1_ovall_peak = pd.merge(df1_overall_wk_peak,df1_overall_month_peak,how="inner",on="Overall")
df1_ovall_peak = df1_ovall_peak.groupby(['Overall']).min().reset_index()
df1_ovall_peak['wave1 - weekid']=(df1_ovall_peak['wave1 - weekid']/2).apply(np.ceil).astype(int)
df1_ovall_peak['wave2 - weekid']=(df1_ovall_peak['wave2 - weekid']/2).apply(np.ceil).astype(int)

In [61]:
df1_ovall_peak

,Overall,wave1 - weekid,wave2 - weekid,wave1 - monthid,wave2 - monthid
0,India,20,54,5,13


In [62]:
df1_stwise_month_1=df1_stwise_month_1.loc[df1_stwise_month_1.groupby("State_Code")["Cases"].idxmax()]

In [63]:
df1_stwise_month_1 = df1_stwise_month_1.rename(columns = {'monthid':'wave1 - monthid'}).drop(columns= ['Cases'])

In [64]:
df1_stwise_month_2=df1_stwise_month_2.loc[df1_stwise_month_2.groupby("State_Code")["Cases"].idxmax()]
df1_stwise_month_2 = df1_stwise_month_2.rename(columns = {'monthid':'wave2 - monthid'}).drop(columns= ['Cases'])


DROPPING STATENAME COLUMNS

In [65]:
df1_sub1 = df1_sub1.drop(columns= ['State_Code'])
df1_sub2 = df1_sub2.drop(columns= ['State_Code'])
df1_month_1 = df1_month_1.drop(columns= ['State_Code'])
df1_month_2 = df1_month_2.drop(columns= ['State_Code'])

In [66]:
df1_district_week = pd.merge(df1_sub1,df1_sub2,how='inner',on='districtid')
df1_district_month = pd.merge(df1_month_1,df1_month_2,how='inner',on='districtid')

MERGING ON DISTRICTID

In [67]:
df1_district = pd.merge(df1_district_week,df1_district_month,how='inner',on='districtid')

In [68]:
df1_stwise_week = pd.merge(df1_stwise_week_1,df1_stwise_week_2,how='inner',on='State_Code')
df1_stwise_month = pd.merge(df1_stwise_month_1,df1_stwise_month_2,how='inner',on='State_Code')

In [69]:
df1_stwise = pd.merge(df1_stwise_week,df1_stwise_month,how='inner',on='State_Code')

In [70]:
df1_stwise = df1_stwise.rename(columns={'State_Code':'stateid'})

In [71]:
df1_district = df1_district.groupby(['districtid']).min().reset_index()
df1_stwise = df1_stwise.groupby(['stateid']).min().reset_index()

APPLY CEILFUNC TO GET  WEEKID BASED ON ODD/EVEN VALUE

In [72]:
df1_stwise['wave1 - weekid']=(df1_stwise['wave1 - weekid']/2).apply(np.ceil).astype(int)
df1_stwise['wave2 - weekid']=(df1_stwise['wave2 - weekid']/2).apply(np.ceil).astype(int)

In [73]:
df1_district['wave1 - weekid']=(df1_district['wave1 - weekid']/2).apply(np.ceil).astype(int)
df1_district['wave2 - weekid']=(df1_district['wave2 - weekid']/2).apply(np.ceil).astype(int)

In [74]:
#WRITING TO CSVS
df1_district.to_csv('district-peaks.csv',index=False)
df1_stwise.to_csv('state-peaks.csv',index=False)
df1_ovall_peak.to_csv('overall-peaks.csv',index=False)


# FINISHED